<a href="https://colab.research.google.com/github/langwnwk11/quant/blob/main/fund_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 基金相关数据获取




# 依赖相关

## 安装依赖库

In [1]:
!pip install -q akshare --upgrade
!pip install -q baostock -i https://pypi.org/simple
!pip install --upgrade baostock -i https://pypi.org/simple
!pip install -q yfinance

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.9 MB/s eta 0:00:00


## 导入公共包

In [2]:
from scipy.stats import pearsonr
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
import akshare as ak
import baostock as bs
from datetime import datetime
import matplotlib.pyplot as plt
import os
# from google.colab import drive
import statsmodels.api as sm
from google.colab import drive
import time # Import the time module
import seaborn as sns # Import seaborn for heatmap
# drive.mount('/content/drive')

#全局配置

##全局变量

### 常量

In [3]:
start_date = '20160101'
end_date = datetime.now().strftime('%Y%m%d')

skip=0

path = '/content/drive/MyDrive/QuantData/fund'

attemp_count=3
retry_delay_seconds=60 # New global variable for retry delay
post_execution_delay_seconds=10 # New global variable for delay after each execution

### 基金代码

In [4]:
fund_list =[{'code':'260112','name':'景顺能源'},
      {'code':'004496','name':'前海开源'},
      # {'code':'510500','name':'中证500ETF'},
      # {'code':'588000','name':'科创50ETF'}, ## 上交所

      # ##深交所
      #  {'code':'159922','name':'中证500ETF'},
      # {'code':'159915','name':'创业板ETF'},
      # {'code':'159901','name':'深证100ETF'}
      ]




### 因子

In [5]:
factor_list=[{'code':'sh000300','name':'沪深300'},

      # {'code':'510500','name':'中证500ETF'},
      # {'code':'588000','name':'科创50ETF'}, ## 上交所

      # ##深交所
      #  {'code':'159922','name':'中证500ETF'},
      # {'code':'159915','name':'创业板ETF'},
      # {'code':'159901','name':'深证100ETF'}
      ]

## 定于公共函数

### 重试装饰器

In [6]:
def retry_on_failure(
    func=None,
    attemp_count_param=None,
    retry_delay_seconds_param=None,
    post_execution_delay_seconds_param=None
):
    """一个通用的装饰器，用于实现函数的重试逻辑。"""
    def decorator(f):
        def wrapper(*args, **kwargs):
            # Use provided parameters or global defaults
            current_attemp_count = attemp_count_param if attemp_count_param is not None else attemp_count
            current_retry_delay_seconds = retry_delay_seconds_param if retry_delay_seconds_param is not None else retry_delay_seconds
            current_post_execution_delay_seconds = post_execution_delay_seconds_param if post_execution_delay_seconds_param is not None else post_execution_delay_seconds

            for attempt in range(current_attemp_count):
                try:
                    result = f(*args, **kwargs)
                    time.sleep(current_post_execution_delay_seconds)
                    return result
                except Exception as e:
                    print(f"Attempt {attempt + 1} failed for {f.__name__} with args {args}, kwargs {kwargs}: {e}")
                    if attempt < current_attemp_count - 1:
                        time.sleep(current_retry_delay_seconds)
                    else:
                        print(f"Failed to execute {f.__name__} after {current_attemp_count} attempts.")
                        raise
        return wrapper

    if func is None:
        return decorator
    else:
        return decorator(func)

### 获取指数收益率 baostock

In [7]:
@retry_on_failure(post_execution_delay_seconds_param=5)
def get_index_baostock(prefix,code):
  lg = bs.login()

  # 详细指标参数，参见“历史行情指标参数”章节；“周月线”参数与“日线”参数不同。
  # 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
  rs = bs.query_history_k_data_plus(prefix+'.'+code,
      "date,code,open,high,low,close,peTTM",
      start_date='1997-01-01', end_date='2322-01-01', frequency="d")

  # 打印结果集
  data_list = []
  while (rs.error_code == '0') & rs.next():
      # 获取一条记录，将记录合并在一起
      data_list.append(rs.get_row_data())
  result = pd.DataFrame(data_list, columns=rs.fields)
  bs.logout()
  return result

### 格式化padas数据集

In [8]:
def format_df(df,column_date,column_close,column_return):
  df['date'] = pd.to_datetime(df[column_date]) # Ensure it's datetime
  df = df.set_index('date').sort_index() # Set date as index
  df[column_close] = pd.to_numeric(df[column_close], errors='coerce') # Convert 'close' to numeric
  df = df[column_close].pct_change().dropna().rename(column_return) # Calcul
  return df

### 获取申万一级行业信息

In [9]:
@retry_on_failure
def get_sw_first_info():
  df=ak.sw_index_first_info()
  return df

### 获取申万一级具体行业数据


In [10]:
@retry_on_failure
def get_sw_first_induesty_data(code):
  df=ak.index_hist_sw(symbol=code, period="day")
  return df


### 黄金数据获取

In [11]:
@retry_on_failure(post_execution_delay_seconds_param=10)
def get_gold(code):
  df= ak.spot_hist_sge(symbol=code)
  return df

### 定义线性回归函数

In [12]:

# 1. 准备数据 (假设你已合并好 df_merged)
# y: 基金超额收益率 (基金日收益 - 无风险利率，简单起见可直接用日收益率)
# X: 因子矩阵 (市场、黄金、商品)

def factor_regression(fund_name, y, X_factors):
    # 为自变量添加常数项 (即 Alpha 截距)
    X = sm.add_constant(X_factors)

    # 构建 OLS 回归模型
    model = sm.OLS(y, X).fit()

    print(f"--- {fund_name} 多因子回归报告 ---")
    print(model.summary())
    return model

# 数据获取


## 因子数据

### 10年国债收益率

In [13]:
bond_10 = ak.bond_zh_us_rate(start_date=start_date)
bond_10
rf = bond_10['中国国债收益率10年'].iloc[-1] / 100
print(rf)

0.018193


In [14]:
bond_10

,日期,中国国债收益率2年,中国国债收益率5年,中国国债收益率10年,中国国债收益率30年,中国国债收益率10年-2年,中国GDP年增率,美国国债收益率2年,美国国债收益率5年,美国国债收益率10年,美国国债收益率30年,美国国债收益率10年-2年,美国GDP年增率
0,2016-01-04,2.4598,2.7488,2.8710,3.6246,0.4112,NaN,1.02,1.73,2.24,2.98,1.22,NaN
1,2016-01-05,2.4944,2.7871,2.8911,3.6446,0.3967,NaN,1.04,1.73,2.25,3.01,1.21,NaN
2,2016-01-06,2.5038,2.7598,2.8861,3.6155,0.3823,NaN,0.99,1.65,2.18,2.94,1.19,NaN
3,2016-01-07,2.4928,2.7538,2.8511,3.5718,0.3583,NaN,0.96,1.61,2.16,2.92,1.20,NaN
4,2016-01-08,2.4790,2.7347,2.8362,3.5537,0.3572,NaN,0.94,1.57,2.13,2.91,1.19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,2026-01-29,1.3908,1.5784,1.8174,2.2800,0.4266,NaN,3.53,3.80,4.24,4.85,0.71,NaN
2688,2026-01-30,1.3765,1.5761,1.8112,2.2890,0.4347,NaN,3.52,3.79,4.26,4.87,0.74,NaN
2689,2026-02-02,1.3852,1.5768,1.8200,2.2810,0.4348,NaN,3.57,3.83,4.29,4.90,0.72,NaN
2690,2026-02-03,1.3772,1.5721,1.8172,2.2800,0.4400,NaN,3.57,3.83,4.28,4.90,0.71,NaN


In [15]:
# 1 + rf 表示一年的总收益因子。
# **(1/252) 表示将这个总收益因子除以252（通常假设一年有252个交易日），从而得到每天的收益因子。
# - 1 是为了从收益因子中减去本金，只保留每日的收益率。
daily_rf = (1 + rf)**(1/252) - 1  # 假设一年有252个交易日
daily_rf

np.float64(7.154814510212937e-05)

### 沪深300

In [16]:
df=get_index_baostock('sh','000300')
df_300=format_df(df,'date','close','沪深300')
df_300

login success!
logout success!


,沪深300
date,
2005-01-05,0.009941
2005-01-06,-0.009460
2005-01-07,0.000797
2005-01-10,0.010083
2005-01-11,0.003276
...,...
2026-01-29,0.007605
2026-01-30,-0.009998
2026-02-02,-0.021324


### 中证1000


In [17]:
# df_1000=get_index_ret('sz000852','中证1000')
# df_1000

df=get_index_baostock('sh','000852')
df
df_1000=format_df(df,'date','close','中证1000')
df_1000

login success!
logout success!


,中证1000
date,
2005-01-05,0.023838
2005-01-06,-0.008009
2005-01-07,0.006241
2005-01-10,0.011760
2005-01-11,0.003440
...,...
2026-01-29,-0.008046
2026-01-30,-0.009283
2026-02-02,-0.033851


### 300价值


In [18]:
df=get_index_baostock('sh','000919')
df_value=format_df(df,'date','close','300价值')
df_value

login success!
logout success!


,300价值
date,
2008-01-22,-0.076954
2008-01-23,0.045881
2008-01-24,0.003864
2008-01-25,0.010689
2008-01-28,-0.074219
...,...
2026-01-29,0.017391
2026-01-30,-0.010399
2026-02-02,-0.015781


### 300成长

In [19]:
df=get_index_baostock('sh','000918')
df_growth=format_df(df,'date','close','300成长')
df_growth

login success!
logout success!


,300成长
date,
2008-01-22,-0.075043
2008-01-23,0.040168
2008-01-24,0.006976
2008-01-25,0.015938
2008-01-28,-0.065858
...,...
2026-01-29,0.007002
2026-01-30,-0.006541
2026-02-02,-0.023481


### SMB (规模因子)
用 中证1000指数 (小盘) 减去 沪深300指数 (大盘)

In [20]:
# 构造 SMB (小盘减大盘)
# 示例：确保日期对齐后再相减
# axis=1：表示将这两个 Series 作为不同的列进行拼接，
# 形成一个 DataFrame。
# join='inner'：这是关键！
# 它确保了只有同时存在于 df_1000 和 df_300 索引（日期）中的数据才会被保留下来。这样，combined_df 中的每一行都对应一个在两个指数中都有数据的日期。
df_1000_300 = pd.concat([df_1000, df_300], axis=1, join='inner')
df_1000_300.columns = ['中证1000', '沪深300']
smb = (df_1000_300['中证1000'] - df_1000_300['沪深300']).rename('SMB')
smb

# # 构造 HML (价值减成长)
# ret_value = get_index_ret("sh000918")  # 沪深300价值
# ret_growth = get_index_ret("sh000919") # 沪深300成长
# hml = ret_value - ret_growth

,SMB
date,
2005-01-05,0.013897
2005-01-06,0.001451
2005-01-07,0.005443
2005-01-10,0.001677
2005-01-11,0.000164
...,...
2026-01-29,-0.015651
2026-01-30,0.000715
2026-02-02,-0.012526


### HML (价值因子)
用 300价值指数 减去 300成长指数。

In [21]:
df_value_growth = pd.concat([df_value, df_growth], axis=1, join='inner')
df_value_growth.columns = ['300价值', '300成长']
hml = (df_value_growth['300价值'] - df_value_growth['300成长']).rename("HML")
hml

,HML
date,
2008-01-22,-0.001911
2008-01-23,0.005712
2008-01-24,-0.003113
2008-01-25,-0.005249
2008-01-28,-0.008361
...,...
2026-01-29,0.010389
2026-01-30,-0.003859
2026-02-02,0.007701


### 行业数据

#### 获取行业代码，批量获取行业数据

In [ ]:
industry_data_dict = {}

sw=get_sw_first_info()
print("遍历行业代码和行业名称:")
for index, row in sw.iterrows():
    industry_code = row['行业代码']
    industry_name = row['行业名称']
    industry_code = industry_code.replace('.SI', '') # Clean the code by removing '.SI'
    print(f"行业代码: {industry_code}, 行业名称: {industry_name}")
    df=get_sw_first_induesty_data(industry_code)
    industry_data_dict[industry_name]=df
industry_data_dict

遍历行业代码和行业名称:
行业代码: 801010, 行业名称: 农林牧渔
行业代码: 801030, 行业名称: 基础化工
行业代码: 801040, 行业名称: 钢铁
行业代码: 801050, 行业名称: 有色金属
行业代码: 801080, 行业名称: 电子
行业代码: 801880, 行业名称: 汽车
行业代码: 801110, 行业名称: 家用电器
行业代码: 801120, 行业名称: 食品饮料
行业代码: 801130, 行业名称: 纺织服饰
行业代码: 801140, 行业名称: 轻工制造
行业代码: 801150, 行业名称: 医药生物
行业代码: 801160, 行业名称: 公用事业
行业代码: 801170, 行业名称: 交通运输
行业代码: 801180, 行业名称: 房地产
行业代码: 801200, 行业名称: 商贸零售
行业代码: 801210, 行业名称: 社会服务
行业代码: 801780, 行业名称: 银行
行业代码: 801790, 行业名称: 非银金融
行业代码: 801230, 行业名称: 综合
行业代码: 801710, 行业名称: 建筑材料
行业代码: 801720, 行业名称: 建筑装饰
行业代码: 801730, 行业名称: 电力设备
行业代码: 801890, 行业名称: 机械设备
行业代码: 801740, 行业名称: 国防军工
行业代码: 801750, 行业名称: 计算机
行业代码: 801760, 行业名称: 传媒


#### 合并行业数据

In [ ]:
industry_returns_list = []
for industry_name, df  in industry_data_dict.items():
    if not df.empty: # Changed from df_1000.empty to df.empty, as df_1000 is not relevant here
        # Rename '日期' column to 'date'
        df.rename(columns={'日期': 'date'}, inplace=True)

        # Convert 'date' to datetime and set as index
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date').sort_index()

        # Calculate daily percentage change of '收盘' price
        returns_series = df['收盘'].pct_change().dropna()

        # Rename the series with the industry name
        returns_series = returns_series.rename(industry_name)

        # Append to the list
        industry_returns_list.append(returns_series)
    else:
        print(f"Warning: DataFrame for industry '{industry_name}' is empty. Skipping return calculation.")


merged_industry = pd.concat(industry_returns_list, axis=1, join='inner')

merged_industry

### 黄金数据获取


In [ ]:
df=get_gold('Au99.99')
gold_df=format_df(df,'date','close','gold')
# gold_df['date'] = pd.to_datetime(gold_df['date']) # Ensure it's datetime
# gold_df = gold_df.set_index('date').sort_index() # Set date as index
# # Assuming return column is 'close' and date is already handled as index
# gold_df['gold'] = gold_df['close'].pct_change()
# gold_df = gold_df['gold'].dropna()
gold_df


### 铜的数据获取

In [ ]:
copper_data = yf.download("HG=F", start='2000-01-01')
copper_data.index = pd.to_datetime(copper_data.index)
copper_data.index.name = 'date'

# Calculate daily returns for copper and create a new DataFrame
copper_df = pd.DataFrame(index=copper_data.index)
copper_df['copper'] = copper_data['Close'].pct_change().dropna()
copper_df

### 合并行业收益率和因子数据

In [ ]:
# merge 沪深300 ，smb ，hml
merge=pd.merge(df_300, smb, left_index=True, right_index=True, how='inner')
merge=pd.merge(merge, hml, left_index=True, right_index=True, how='inner')

# merge 行业数据
merge = pd.merge(merged_industry, merge, left_index=True, right_index=True, how='inner')
# merge 黄金 铜
merge = pd.merge(merge,gold_df , left_index=True, right_index=True, how='inner')
merge = pd.merge(merge,copper_df , left_index=True, right_index=True, how='inner')
merge

## 基金数据获取

In [ ]:
target_data_dict = {}

# Create directory if it doesn't exist for saving data
if not os.path.exists(path):
    os.makedirs(path)


if skip == 0:
    print("skip is 0, fetching and saving data...")
    for target in target_list:
        code = target['code']
        name = target['name']
        file_name = f"{code}.csv"
        full_path = os.path.join(path, file_name)
        if os.path.exists(full_path):
          df = pd.read_csv(full_path)
          target_data_dict[name] = df
        else:
          try:
              # df = ak.fund_etf_hist_em(symbol=code, period="daily", start_date=start_date, end_date=end_date, adjust="hfq")
              df = ak.fund_open_fund_info_em(symbol=code, indicator="累计净值走势")
              target_data_dict[name] = df # Still add to dict for subsequent correlation calculation

              # Save data immediately after fetching
              file_name = f"{code}.csv"
              full_path = os.path.join(path, file_name)
              df.to_csv(full_path, index=False, encoding='utf-8')
              print(f"Successfully fetched and saved data for {name} to {full_path}.")
              time.sleep(10) # Add a 10-second delay
          except Exception as e:
              print(f"Error fetching or saving data for {name} ({code}): {e}")

else:
    print("skip is not 0, loading data from files...")
    for target in target_list:
        code = target['code']
        name = target['name']
        file_name = f"{code}.csv"
        full_path = os.path.join(path, file_name)
        if os.path.exists(full_path):
            try:
                df = pd.read_csv(full_path)
                target_data_dict[name] = df
                print(f"Successfully loaded data for {name} from {full_path}.")
            except Exception as e:
                print(f"Error loading data for {name} from {full_path}: {e}")
        else:
            print(f"Data file not found for {name} at {full_path}. Skipping.")

In [ ]:
stock_zh_index_daily_df = ak.stock_zh_index_daily(symbol="sz000823")
stock_zh_index_daily_df

In [ ]:
index_zh_a_hist_df = ak.index_zh_a_hist(symbol="000823", period="daily", start_date="19700101", end_date="22220101")
index_zh_a_hist_df

In [ ]:
stock_zh_index_value_csindex_df = ak.stock_zh_index_value_csindex(symbol="H30374")
stock_zh_index_value_csindex_df

In [ ]:
import akshare as ak

# 获取单只股票历史估值
df = ak.stock_zh_index_value_csindex(symbol="H30016")
df

In [ ]:
sw_index_first_info_df = ak.sw_index_first_info()
print(sw_index_first_info_df)

In [ ]:
szse_df = ak.stock_szse_summary()
szse_df

In [ ]:
stock_szse_summary_df = ak.stock_szse_summary(date="20260202")
stock_szse_summary_df

In [ ]:
stock_sse_deal_daily_df = ak.stock_sse_deal_daily(date="20260202")
stock_sse_deal_daily_df